<a href="https://colab.research.google.com/github/seacabezasma/IIS_2020_01_Taller_01/blob/master/Taller_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1
### [Introducción a los sistemas inteligentes 2020-1](https://fagonzalezo.github.io/iis-2020-1/)

Integrantes del grupo (máximo 3):

* Nombre_1 ID_1
* Nombre_2 ID_2
* Nombre_3 ID_3

___________

**Fecha límite de entrega**: _Lunes 13 de Abril_ antes de la medianoche 
Envios fuera de este plazo no se tendrán en cuenta.


Cerciórese de reiniciar y correr el notebook en su totalidad antes de enviarlo. Verifique que todas las salidas se muestran de manera correcta.

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/OapI3DaGbxdtWoUkpLiR) antes de la hora límite. El archivo debe nombrarse como  `isi-taller1-unalusername1-unalusername2-unalusername3.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).

___________

## Cuadrado de Rubik (o Cubo de Rubik 2D)

El *Cuadrado de Rubik* es un rompecabezas inspirado en el famoso cubo de Rubik. Consiste de un arreglo de 16 fichas organizadas en una matriz de $4\times 4$ como se ilustra en la siguiente figura:

<img src="https://raw.githubusercontent.com/fagonzalezo/iis-2018-2/master/rubik2D.png" 
alt="Cuadrado de Rubik " width="240" height="180" border="10" />

Los colores son ilustrativos, lo importante es el número en cada una de las fichas. Se pueden hacer 10 movimientos diferentes correspondientes a rotar las 4 fichas alrededor de cada uno de los puntos A, B, C, D y E en el sentido de las manecillas del reloj o en el sentido opuesto.

Su objetivo es modelar el *Cuadrado de Rubik* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Cuadrado de Rubik

Un Cuadrado de Rubik debe representarse como una lista con valores enteros que representan cada una de las fichas.

Por ejemplo un Cuadrado de Rubik resuelto debe verse así:

```python
[ 1,  2,  3,  4,
  5,  6,  7,  8,
  9, 10, 11, 12,
 13, 14, 15, 16]
```

#### Definición de acciones

La siguiente lista define las posibles acciones que se pueden ejecutar:

In [0]:
'''
These values MUST not be changed.
They represent the movements of the Rubik's Square.
'''
actions = ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

Cada acción indica la posición y el sentido. Por ejemplo, `'C-'` rota la posición C en el sentido opuesto de las manecillas del reloj. Si aplicamos esta acción al estado solución se obtiene el estado:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 10, 12,
 13, 14, 15, 16]
```

Si sobre este estado, aplicamos la acción `'E+'` obtenemos:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 15, 10,
 13, 14, 16, 12]
```

#### Clase Rubik2D_problem

In [0]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

    
class rubik2d_problem(Problem):    
    
    def __init__(self, initial):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15]
        [1, 9, 4, 8, 5, 6, 3, 2, 15, 10, 11, 12, 13, 14, 7, 16]
        [2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12]
        '''
        self.expanded = 0
        # Your code here
        pass
    
    def actions(self, state):
        """Return a list of actions that can be executed in the given
        state."""
        # Your code here
        pass

    def result(self, state, action):
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        # Your code here
        pass

    def is_goal(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        # Your code here
        pass


    def action_cost(self, s, a, s1): 
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        # Your code here
        pass

#==============================================================
# PRUEBA MOVIMIENTOS
rubik = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

def rotar(cubo_original, cubo_copia, movimiento):
    
    #===============================
    # GIRA EN TORNO A 
    if movimiento == 'A+':
        cubo_original[0]=cubo_copia[4]
        cubo_original[1]=cubo_copia[0]
        cubo_original[4]=cubo_copia[5]
        cubo_original[5]=cubo_copia[1]
        
    elif movimiento == 'A-':        
        cubo_original[0]=cubo_copia[1]        
        cubo_original[1]=cubo_copia[5]
        cubo_original[4]=cubo_copia[0]
        cubo_original[5]=cubo_copia[4]
    
        
    #===============================
    # GIRA EN TORNO B
    elif movimiento == 'B+':
        cubo_original[2]=cubo_copia[6]
        cubo_original[3]=cubo_copia[2]
        cubo_original[6]=cubo_copia[7]
        cubo_original[7]=cubo_copia[3]        
        
    elif movimiento == 'B-':
        cubo_original[2]=cubo_copia[3]
        cubo_original[3]=cubo_copia[7]
        cubo_original[6]=cubo_copia[2]
        cubo_original[7]=cubo_copia[6]

    #===============================
    # GIRA EN TORNO C
    elif movimiento == 'C+':
        cubo_original[5]=cubo_copia[9]
        cubo_original[6]=cubo_copia[5]
        cubo_original[9]=cubo_copia[10]
        cubo_original[10]=cubo_copia[6]        
        
    elif movimiento == 'C-':
        cubo_original[5]=cubo_copia[6]
        cubo_original[6]=cubo_copia[10]
        cubo_original[9]=cubo_copia[5]
        cubo_original[10]=cubo_copia[9]

    #===============================
    # GIRA EN TORNO D
    elif movimiento == 'D+':
        cubo_original[8]=cubo_copia[12]
        cubo_original[9]=cubo_copia[8]
        cubo_original[12]=cubo_copia[13]
        cubo_original[13]=cubo_copia[9]        
        
    elif movimiento == 'D-':
        cubo_original[8]=cubo_copia[9]
        cubo_original[9]=cubo_copia[13]
        cubo_original[12]=cubo_copia[8]
        cubo_original[13]=cubo_copia[12]

    #===============================
    # GIRA EN TORNO E
    elif movimiento == 'E+':
        cubo_original[10]=cubo_copia[14]
        cubo_original[11]=cubo_copia[10]
        cubo_original[14]=cubo_copia[15]
        cubo_original[15]=cubo_copia[11]        
        
    elif movimiento == 'E-':
        cubo_original[10]=cubo_copia[11]
        cubo_original[11]=cubo_copia[15]
        cubo_original[14]=cubo_copia[10]
        cubo_original[15]=cubo_copia[14]

# ==================================================
print(rubik)# el original
rotar(rubik, rubik.copy(), 'C-')
print(rubik)# como quedo
rotar(rubik, rubik.copy(), 'E+')
print(rubik)# como quedo
#===================================================


### 2. Evalue su código con diferentes estrategias de búsqueda



In [0]:
def bfs(problem):
    ### your code here ###
    return []

def iterativeDeepeningSearch(problem):
    ### your code here ###
    return []

problem = rubik2d_problem( [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15])

actions = bfs(problem)
print(actions)
actions = iterativeDeepeningSearch(problem)
print(actions)



[]
[]


Evaluelo para ver cuál es la máxima profundidad que se puede alcanzar en un tiempo razonable con cada estrategia de búsqueda. Reporte los resultados.

### 3. Implemente diferente heurísticas para el problema

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de [Russell10].

In [0]:
def nullHeuristic(state):
    return 0

def myHeuristic(state):
    ### your code here ###
    return 0

def aStarSearch(problem, h=nullHeuristic):
    ### your code here ###
    return []



In [0]:
problem = rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print(iterativeDeepeningSearch(problem))
print(aStarSearch(problem, myHeuristic))

[]
[]
